In [1]:
import oracledb
import cx_Oracle
from datetime import datetime, timedelta

In [2]:
try:
    cx_Oracle.init_oracle_client(lib_dir=r"F:\ruslan\dev\oracle\instantclient_19_9")
except:
    pass


In [3]:
conn_nwp = oracledb.connect(user="nwp", password="modeli", dsn="192.168.211.62/serv")
cursor_nwp = conn_nwp.cursor()

cursor_nwp.execute("""
/* SELECT to_char(DATAS, 'YYYY-MM-DD'), ADVANCE_TIME,  TEMP */
SELECT DATAS, ADVANCE_TIME,  TEMP
FROM NWP.GROUND_DATA
WHERE CITY_ID = 936 
      AND model_id = 5
      AND ADVANCE_TIME between 0 and 48
      AND datas >= '01-12-2023'
      AND to_char(datas, 'HH24') = '00'
ORDER BY datas, advance_time
""")
date = None
forecast_arr = []
f_row = [] 
for c in cursor_nwp:
    # Нова дата - создаем новую строку данных
    if date != c[0]:
        # Если дата пройдена, присоединить строку данных f_row к массиву forecast_arr
        if date:
            forecast_arr.append(f_row) 
        # Инициализируем следующую дату и строку данных
        date = c[0]       
        f_row = [None for _ in range(50)]
        f_row[0] = c[0] # Дату c[0] записать в позицию 0
    # В позицию списка, равную заблаговременности c[1] записать температуру c[2]
    f_row[c[1] + 1] = c[2]
# Присоединить последнюю строку данных f_row к массиву forecast_arr
forecast_arr.append(f_row)        
print(*forecast_arr, sep='\n')
conn_nwp.close()

[datetime.datetime(2023, 12, 1, 0, 0), -4.69, -4.95, -4.75, -4.77, -4.38, -4.07, -3.96, -3.92, -3.53, -2.83, -2.47, -2.32, -2.41, -2.85, -3.72, -4.17, -5.38, -6.47, -7.1, -7.53, -7.81, -7.96, -7.68, -7.82, -7.82, -7.06, -6.4, -5.88, -5.51, -5.28, -5.02, -4.84, -4.27, -3.57, -3.01, -2.8, -2.79, -2.94, -3.36, -3.85, -4.26, -4.4, -4.45, -4.54, -4.65, -4.81, -4.95, -5.06, -5.14]
[datetime.datetime(2023, 12, 2, 0, 0), -6.47, -6.16, -5.8, -5.54, -5.42, -5.32, -5.4, -5.37, -4.91, -4.17, -3.46, -3.02, -3.0, -3.06, -3.33, -3.62, -3.97, -4.16, -4.39, -4.45, -4.53, -4.63, -4.77, -4.97, -5.11, -5.29, -5.46, -5.64, -5.71, -5.71, -5.81, -5.92, -5.74, -5.52, -5.35, -5.12, -5.24, -5.57, -5.97, -6.43, -6.41, -6.39, -6.22, -6.08, -5.79, -5.69, -5.69, -5.65, -5.63]
[datetime.datetime(2023, 12, 3, 0, 0), -5.1, -5.24, -5.3, -5.43, -5.47, -5.47, -5.56, -5.64, -5.47, -5.37, -5.3, -5.29, -5.46, -5.93, -6.61, -7.11, -7.29, -7.27, -7.2, -7.31, -7.45, -7.72, -8.1, -8.84, -9.54, -9.92, -10.12, -9.98, -9.53, -9.14

In [4]:
conn_gts = cx_Oracle.connect(user="cligts", password="belorgmc", dsn="192.168.211.173/clidb")
cursor_gts = conn_gts.cursor()

cursor_gts.execute(
"""
SELECT DATE_OBS, TEMPDB
FROM gts.synop
WHERE date_obs >= '25-12-2023' AND station_id = '26850'
""")

gts_rows = cursor_gts.fetchall()
for c in gts_rows:
    print(c)

conn_gts.close()

(datetime.datetime(2023, 12, 25, 0, 0), -2.1)
(datetime.datetime(2023, 12, 25, 3, 0), -2.0)
(datetime.datetime(2023, 12, 25, 6, 0), 0.0)
(datetime.datetime(2023, 12, 25, 9, 0), 3.6)
(datetime.datetime(2023, 12, 25, 12, 0), 4.9)
(datetime.datetime(2023, 12, 25, 15, 0), 3.8)
(datetime.datetime(2023, 12, 25, 18, 0), 4.2)
(datetime.datetime(2023, 12, 25, 21, 0), 3.7)
(datetime.datetime(2023, 12, 26, 0, 0), 3.1)
(datetime.datetime(2023, 12, 26, 3, 0), 3.1)
(datetime.datetime(2023, 12, 26, 6, 0), 2.2)
(datetime.datetime(2023, 12, 26, 9, 0), 2.8)
(datetime.datetime(2023, 12, 26, 12, 0), 2.4)
(datetime.datetime(2023, 12, 26, 15, 0), 0.6)
(datetime.datetime(2023, 12, 26, 18, 0), 2.3)
(datetime.datetime(2023, 12, 26, 21, 0), 4.5)
(datetime.datetime(2023, 12, 27, 0, 0), 3.3)
(datetime.datetime(2023, 12, 27, 3, 0), 2.1)
(datetime.datetime(2023, 12, 27, 6, 0), 1.4)
(datetime.datetime(2023, 12, 27, 9, 0), 1.4)
(datetime.datetime(2023, 12, 27, 12, 0), 1.2)
(datetime.datetime(2023, 12, 27, 15, 0), 1.2

In [6]:
# Формирование y - значений фактической температуры, соответствующих каждой дате на 48 часов вперед
# td48 = timedelta(hours=48)
# td03 = timedelta(hours=3)
# Поиск каждой совпадающей строки по дате и сроку 00
ind = 0
start = 0
fact_arr = []
for f in forecast_arr:
    date_nwp = f[0]
    for i, g in enumerate(gts_rows[start:]):
        if g[0] == f[0]: # Строка найдена
            # start += i
            break
    # Инициализировать строку данных для фактической температуры 17 значений
    g_row = [None for _ in range(18)]
    g_row[0] = g[0]
    for j in range(17):
        gts_date = gts_rows[start + j][0]
        gts_temperature = gts_rows[start + j][1]
        td = gts_date - date_nwp
        # В пределах 48 часов
        if td <= 48:
            cell = td / 3
            g_row[cell + 1] = gts_temperature
        else:
            start += i
            break
    fact_arr.append(g_row)

print(fact_arr)




TypeError: '<=' not supported between instances of 'datetime.timedelta' and 'int'

In [ ]:
d1 = datetime(2023,12,26)
d1

In [ ]:
td = timedelta(hours=48)
d2 = d1 + td
d2

In [ ]:
t1 = datetime(2023,12,26)
t2 = datetime(2023,12,26)
t3 = t1 + td
print(str(t3))